In [1]:
from completeRun import featureClf,cv_feature
from preamble import *
from LocalDatasets import saveDict,readDict,ScoresFromPredictions,read_features, read_duration,checkForExist,read_did_pred,ScoresAveraging,DoneFeatureMan
import seaborn as sns
from copy import copy
from Noise2 import CIIntervals
plt.rcParams['savefig.dpi'] = 200
from preamble import *
Cat = [20, 21, 26, 333, 334, 335,40668, 4135, 4534, 469, 46, 50]

amountList = [0.25,0.5,0.75,1]
NonCat = [1038,1043,1046,1049,1050,1063,1067,1068,1120,1176,11,12,1459,1462,1464,1466,1467,1468,1475,1476,1478,1479,1485,1487]
second = [1489, 1491, 1492, 1493, 1494, 1497, 14, 1501, 1504, 1510, 1515, 1570, 16, 18, 22, 28, 300, 30, 32, 36, 375, 37, 39,40499,40509,40, 4134, 41, 44, 4538, 458, 53, 54]
for i in second:
    NonCat.append(i)
          
cv = 10
clfNames = [ 'RandomForestClassifier','KNeighborsClassifier', '1NeighborsClassifier', 'SGDClassifier', 'AdaBoost', 'SVC-rbf','GaussianNB', 'BernoulliNB','GradientBoost']
clfNames = [ 'RandomForestClassifier','KNeighborsClassifier', '1NeighborsClassifier', 'SGDClassifier', 'AdaBoost', 'SVC-rbf','GaussianNB', 'BernoulliNB','GradientBoost']


In [2]:
from LocalDatasets import read_featuresFull
#Three types of feature addition
func = 'cvScoreFeatures6Laptop'
func2 = 'cvScoreFeatures5Duration'
func3 = 'cvScoreFeatures4Duration'
clfName = clfNames[0]
amountList = [1.0]
funcMan = DoneFeatureMan(func,clfName,amountList)
func2Man = DoneFeatureMan(func2,clfName,amountList)
func3Man = DoneFeatureMan(func3,clfName,amountList)
intersect = []
for did in funcMan:
    if did in func2Man:
        if did in func3Man:
            intersect.append(did)
didList = []
for did in intersect:
    if readDict(did)['NumberOfSymbolicFeatures'] == 1:
        didList.append(did)

        
less1000 = []
great5000 = []
rest = []
for i,item in enumerate(didList):
    if readDict(item)['NumberOfInstances'] < 1000:
        less1000.append(item)
    elif readDict(item)['NumberOfInstances'] > 5000:
        great5000.append(item)
    else:
        rest.append(item)
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(less1000):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1 = []
        score2 = []
        score3 = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1)
        scores2[i].append(score2)
        scores3[i].append(score3)
score2 = []
score3 = []
score4 = []
for i in clfNames:
    score2.append([[] for i in range(len(amountList)+1)])
    score3.append([[] for i in range(len(amountList)+1)])
    score4.append([[] for i in range(len(amountList)+1)])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                score2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                score2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                score3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                score3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                score4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                score4[j][i+1].append(temp)

scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(rest):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreD2 = []
scoreID3 = []
scoreR4 = []
for i in clfNames:
    scoreD2.append([[] for i in range(len(amountList)+1)])
    scoreID3.append([[] for i in range(len(amountList)+1)])
    scoreR4.append([[] for i in range(len(amountList)+1)])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                scoreD2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                scoreD2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                scoreID3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                scoreID3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                scoreR4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                scoreR4[j][i+1].append(temp)           
            

            
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(great5000):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreGD2 = []
scoreGID3 = []
scoreGR4 = []
ratio = []
for i in clfNames:
    scoreGD2.append([[] for i in range(len(amountList)+1)])
    scoreGID3.append([[] for i in range(len(amountList)+1)])
    scoreGR4.append([[] for i in range(len(amountList)+1)])
    ratio.append([])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                scoreGD2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                scoreGD2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                scoreGID3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                scoreGID3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                scoreGR4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                scoreGR4[j][i+1].append(temp)
            
for i,x in enumerate(amountList):
    amountList[i] = x*100
amountList2 = copy(amountList)
amountList.insert(0,0)
x_axis = amountList

In [3]:
loss = []
for k,temp in enumerate([score4,scoreR4,scoreGR4]):
    loss.append([])
    for i,item in enumerate(temp):
        loss[k].append(100*(sum(temp[i][0])/len(temp[i][0])-sum(temp[i][1])/len(temp[i][1])))
arg = []
for i,clfName in enumerate(clfNames):
    arg.append([])
    for k, item in enumerate(loss):
        arg[i].append(item[i])
stds = []
for i,item in enumerate(clfNames):
    stds.append([])
    for k in range(3):
        stds[i].append([])
for k,temp in enumerate([score4,scoreR4,scoreGR4]):
    for i,item in enumerate(temp):#clf
        for j,itemz in enumerate(temp[i][0]):
            stds[i][k].append(temp[i][0][j]-temp[i][1][j])
diffI = []
for i,item in enumerate(stds):
    diffI.append([])
    for k,itemz in enumerate(stds[i]):
        diffI[i].append(100*np.array(stds[i][k]).std())

In [4]:
diffI

[[7.5672734110126854, 4.5152507319536541, 4.4786589304208722],
 [7.6019093700640452, 21.18810131013533, 11.124755913777276],
 [7.7855619967386485, 22.417049765666182, 12.885569453913181],
 [15.632098667656134, 15.482727232096313, 7.9897202792035689],
 [4.4969806512790385, 3.858833320447713, 3.4118181827863285],
 [5.2222792253434083, 6.5650846827490197, 5.895513010258294],
 [2.1887662037808546, 0.75592334726093013, 0.28042139110852182],
 [3.7449221497078393, 1.1626339359676352, 0.074991923965503832],
 [4.3539719192855602, 4.441926084895421, 0.85189192263306202]]

In [5]:
from LocalDatasets import read_featuresFull
#Three types of feature addition
func = 'cvScoreFeatures6Laptop'
func2 = 'cvScoreFeatures5Duration'
func3 = 'cvScoreFeatures4Duration'
clfName = clfNames[0]
amountList = [1.0]
funcMan = DoneFeatureMan(func,clfName,amountList)
func2Man = DoneFeatureMan(func2,clfName,amountList)
func3Man = DoneFeatureMan(func3,clfName,amountList)
intersect = []
for did in funcMan:
    if did in func2Man:
        if did in func3Man:
            intersect.append(did)
didList = []
for did in intersect:
    if readDict(did)['NumberOfSymbolicFeatures'] == 1:
        didList.append(did)
        

less20 = []
great100 = []
rest2 = []
for i,item in enumerate(didList):
    if readDict(item)['NumberOfFeatures'] < 20:
        less20.append(item)
    elif readDict(item)['NumberOfFeatures'] > 100:
        great100.append(item)
    else:
        rest2.append(item)
        

scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(less20):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1 = []
        score2 = []
        score3 = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1)
        scores2[i].append(score2)
        scores3[i].append(score3)
score2 = []
score3 = []
score4 = []
for i in clfNames:
    score2.append([[] for i in range(len(amountList)+1)])
    score3.append([[] for i in range(len(amountList)+1)])
    score4.append([[] for i in range(len(amountList)+1)])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                score2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                score2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                score3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                score3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                score4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                score4[j][i+1].append(temp)

scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(rest2):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreD2 = []
scoreID3 = []
scoreR4 = []
for i in clfNames:
    scoreD2.append([[] for i in range(len(amountList)+1)])
    scoreID3.append([[] for i in range(len(amountList)+1)])
    scoreR4.append([[] for i in range(len(amountList)+1)])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                scoreD2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                scoreD2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                scoreID3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                scoreID3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                scoreR4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                scoreR4[j][i+1].append(temp)           
            

            
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(great100):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreGD2 = []
scoreGID3 = []
scoreGR4 = []
ratio = []
for i in clfNames:
    scoreGD2.append([[] for i in range(len(amountList)+1)])
    scoreGID3.append([[] for i in range(len(amountList)+1)])
    scoreGR4.append([[] for i in range(len(amountList)+1)])
    ratio.append([])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                scoreGD2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                scoreGD2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                scoreGID3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                scoreGID3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                scoreGR4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                scoreGR4[j][i+1].append(temp)
            
for i,x in enumerate(amountList):
    amountList[i] = x*100
amountList2 = copy(amountList)
amountList.insert(0,0)
x_axis = amountList

In [6]:
lossF = []
for k,temp in enumerate([score4,scoreR4,scoreGR4]):
    lossF.append([])
    for i,item in enumerate(temp):
        lossF[k].append(100*(sum(temp[i][0])/len(temp[i][0])-sum(temp[i][1])/len(temp[i][1])))
argF = []
for i,clfName in enumerate(clfNames):
    argF.append([])
    for k, item in enumerate(lossF):
        argF[i].append(item[i])
stds = []
for i,item in enumerate(clfNames):
    stds.append([])
    for k in range(3):
        stds[i].append([])
for k,temp in enumerate([score4,scoreR4,scoreGR4]):
    for i,item in enumerate(temp):#clf
        for j,itemz in enumerate(temp[i][0]):
            stds[i][k].append(temp[i][0][j]-temp[i][1][j])
diffF = []
for i,item in enumerate(stds):
    diffF.append([])
    for k,itemz in enumerate(stds[i]):
        diffF[i].append(100*np.array(stds[i][k]).std())

In [7]:
diffF

[[5.6920268884260237, 3.8612174420856364, 7.5027877391914908],
 [5.9521963183653446, 21.248829693147513, 8.0680659084523523],
 [5.8864452705352797, 22.69240502825846, 9.9508950674952334],
 [12.653951628515889, 15.91764424694353, 10.665290913248395],
 [3.7439618821581186, 3.3348137891198415, 5.3612901796074564],
 [4.7818925879989189, 7.4040265890409778, 4.0115100215250408],
 [1.8180238061723379, 1.002121935878034, 0.5676918081707002],
 [0.94596061380104524, 1.1727534675302391, 4.303165190261204],
 [3.4577676318291597, 4.2816066219380016, 3.3458643556636436]]

In [8]:
#Three types of feature addition
func = 'cvScoreFeatures6Laptop'
func2 = 'cvScoreFeatures5Duration'
func3 = 'cvScoreFeatures4Duration'
clfName = clfNames[0]
amountList = [1.0]
funcMan = DoneFeatureMan(func,clfName,amountList)
func2Man = DoneFeatureMan(func2,clfName,amountList)
func3Man = DoneFeatureMan(func3,clfName,amountList)
intersect = []
for did in funcMan:
    if did in func2Man:
        if did in func3Man:
            intersect.append(did)
didList = []
for did in intersect:
    if did in NonCat:
        didList.append(did)
# didList = intersect

# didList = Cat
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(didList):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1 = []
        score2 = []
        score3 = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3.append(read_features('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3.append(read_features(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1.append(read_features(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2.append(read_features(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1)
        scores2[i].append(score2)
        scores3[i].append(score3)
score1 = []
score2 = []
score3 = []
score4 = []
ratio = []
for i in clfNames:
    score1.append([0 for i in range(len(amountList))])
    score2.append([0 for i in range(len(amountList)+1)])
    score3.append([0 for i in range(len(amountList)+1)])
    score4.append([0 for i in range(len(amountList)+1)])
    ratio.append([])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            score1[j][i] = score1[j][i] + (scores[k][j][i][0])/len(didList)
            score2[j][0] = score2[j][0] + (scores[k][j][i][0])/(len(didList)*len(amountList))
            score2[j][i+1] = score2[j][i+1] + (scores[k][j][i][1])/len(didList)
            score3[j][0] = score3[j][0] + (scores2[k][j][i][0])/(len(didList)*len(amountList))
            score3[j][i+1] = score3[j][i+1] + (scores2[k][j][i][1])/len(didList)
            score4[j][0] = score4[j][0] + (scores3[k][j][i][0])/(len(didList)*len(amountList))
            score4[j][i+1] = score4[j][i+1] + (scores3[k][j][i][1])/len(didList)
for i,x in enumerate(amountList):
    amountList[i] = x*100
amountList2 = copy(amountList)
amountList.insert(0,0)
x_axis = amountList

In [9]:
#Three types of feature addition
func = 'cvScoreFeatures6Laptop'
func2 = 'cvScoreFeatures5Duration'
func3 = 'cvScoreFeatures4Duration'
clfName = clfNames[0]
amountList = [1.0]
funcMan = DoneFeatureMan(func,clfName,amountList)
func2Man = DoneFeatureMan(func2,clfName,amountList)
func3Man = DoneFeatureMan(func3,clfName,amountList)
intersect = []
for did in funcMan:
    if did in func2Man:
        if did in func3Man:
            intersect.append(did)
didList = []
for did in intersect:
    if did in NonCat:
        didList.append(did)
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(didList):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_featuresFull('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_featuresFull(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_featuresFull(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_featuresFull(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreGD2 = []
scoreGID3 = []
scoreGR4 = []
ratio = []
for i in clfNames:
    scoreGD2.append([[] for i in range(len(amountList)+1)])
    scoreGID3.append([[] for i in range(len(amountList)+1)])
    scoreGR4.append([[] for i in range(len(amountList)+1)])
    ratio.append([])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            for temp in scores[k][j][i][0]:
                scoreGD2[j][0].append(temp)
            for temp in scores[k][j][i][1]:
                scoreGD2[j][i+1].append(temp)
            for temp in scores2[k][j][i][0]:
                scoreGID3[j][0].append(temp)
            for temp in scores2[k][j][i][1]:
                scoreGID3[j][i+1].append(temp)
            for temp in scores3[k][j][i][0]:
                scoreGR4[j][0].append(temp)
            for temp in scores3[k][j][i][1]:
                scoreGR4[j][i+1].append(temp)

In [10]:
lossA = []
for k,temp in enumerate([scoreGR4]):
    lossA.append([])
    for i,item in enumerate(temp):
        lossA[k].append(100*(sum(temp[i][0])/len(temp[i][0])-sum(temp[i][1])/len(temp[i][1])))
argA = []
for i,clfName in enumerate(clfNames):
    argA.append([])
    for k, item in enumerate(lossA):
        argA[i].append(item[i])
stds = []
for i,item in enumerate(clfNames):
    stds.append([])
    for k in range(1):
        stds[i].append([])
for k,temp in enumerate([scoreGR4]):
    for i,item in enumerate(temp):#clf
        for j,itemz in enumerate(temp[i][0]):
            stds[i][k].append(temp[i][0][j]-temp[i][1][j])
diffA = []
for i,item in enumerate(stds):
    diffA.append([])
    for k,itemz in enumerate(stds[i]):
        diffA[i].append(100*np.array(stds[i][k]).std())

In [11]:
argA

[[3.083015681084744],
 [7.458952760932958],
 [8.758083415167528],
 [1.7601445621181488],
 [0.8842218124345935],
 [0.6442230275215843],
 [-0.05022133292996722],
 [0.4147562260143234],
 [1.5416010474246455]]

In [7]:
diff = []
for i,item in enumerate(score3):
    diff.append(100*(score3[i][0]-score3[i][4]))

In [8]:
arg2 = []
for i,clfName in enumerate(clfNames):
    arg2.append([])
    arg2[i].append(diff[i])
for i,item in enumerate(arg):
    for k,temp in enumerate(item):
        arg2[i].append(temp)
for i,item in enumerate(argF):
    for k,temp in enumerate(item):
        arg2[i].append(temp)
for i,item in enumerate(arg2):
    for k,itemz in enumerate(arg2[i]):
        arg2[i][k] = round(100*itemz)/100
df = pd.DataFrame(np.array(arg2),
                  columns=['average','small Instances', 'medium Instances', 'large Instances','small Features', 'medium Features', 'large Features'],index=clfNames)
df.round

df.astype(object)
df.to_csv('DDuplicate.csv')

In [9]:
df.astype(object)

,average,small Instances,medium Instances,large Instances,small Features,medium Features,large Features
RandomForestClassifier,24,16,22,33,21,22,28
KNeighborsClassifier,19,16,18,28,19,21,19
1NeighborsClassifier,21,16,20,34,22,22,21
SGDClassifier,13,9.2,13,21,15,12,14
AdaBoost,12,16,11,13,15,13,11
SVC-rbf,13,6.1,12,25,14,14,9.6
GaussianNB,32,32,29,39,33,31,34
BernoulliNB,11,5.5,12,15,6.5,13,11
GradientBoost,16,9.4,16,22,17,14,15


In [41]:
arg2 = []
for i,clfName in enumerate(clfNames):
    arg2.append([])
    for k,temp in enumerate(argA[i]):
        arg2[i].append(temp)
#     arg2[i].append(diff[i])
for i,item in enumerate(arg):
    for k,temp in enumerate(item):
        arg2[i].append(temp)
for i,item in enumerate(argF):
    for k,temp in enumerate(item):
        arg2[i].append(temp)
for i,item in enumerate(arg2):
    for k,itemz in enumerate(arg2[i]):
        arg2[i][k] = int(round(100*itemz)/100)
df = pd.DataFrame(np.array(arg2),
                  columns=['average','small Instances', 'medium Instances', 'large Instances','small Features', 'medium Features', 'large Features'],index=clfNames)
df.round

df.astype(object)

,average,small Instances,medium Instances,large Instances,small Features,medium Features,large Features
RandomForestClassifier,22,15,22,33,21,22,26
KNeighborsClassifier,19,15,17,27,17,21,16
1NeighborsClassifier,21,15,19,32,20,23,18
SGDClassifier,13,8,12,21,14,13,11
AdaBoost,13,14,11,12,13,13,10
SVC-rbf,13,4,12,25,12,15,7
GaussianNB,32,31,28,39,32,31,33
BernoulliNB,10,5,11,13,6,12,10
GradientBoost,15,9,15,22,17,14,12


In [40]:
diff2 = []
for i,clfName in enumerate(clfNames):
    diff2.append([])
    for k,temp in enumerate(diffA[i]):
        diff2[i].append(temp)
#     arg2[i].append(diff[i])
for i,item in enumerate(diffI):
    for k,temp in enumerate(item):
        diff2[i].append(temp)
for i,item in enumerate(diffF):
    for k,temp in enumerate(item):
        diff2[i].append(temp)
for i,item in enumerate(diff2):
    for k,itemz in enumerate(diff2[i]):
        diff2[i][k] = int(round(100*itemz)/100)
df = pd.DataFrame(np.array(diff2),
                  columns=['average','small Instances', 'medium Instances', 'large Instances','small Features', 'medium Features', 'large Features'],index=clfNames)
df.round

df.astype(object)

,average,small Instances,medium Instances,large Instances,small Features,medium Features,large Features
RandomForestClassifier,15,12,14,16,16,14,16
KNeighborsClassifier,17,16,17,16,16,17,18
1NeighborsClassifier,19,17,18,17,20,18,19
SGDClassifier,21,17,23,21,22,23,15
AdaBoost,12,13,11,13,13,12,9
SVC-rbf,19,9,19,24,19,21,13
GaussianNB,27,20,28,30,26,28,25
BernoulliNB,13,8,15,14,9,15,14
GradientBoost,13,10,12,14,15,11,12


In [43]:
tutal = []
for i,item in enumerate(diff2):
    tutal.append([])
    for k,itemz in enumerate(diff2[i]):
        if arg2[i][k] == '-':
            tutal[i].append('-')
        else:
            tutal[i].append(str(arg2[i][k]) + '(+-' + str(diff2[i][k]) + ')')

In [38]:
df = pd.DataFrame(np.array(diff2),
                  columns=['avg','S I', 'M I', 'L I','S I', 'M F', 'L F'],index=clfNames)
df.round
df.astype(object)
# df.to_csv('Noise.csv')

,avg,S I,M I,L I,S I,M F,L F
RandomForestClassifier,16,13,15,16,17,14,17
KNeighborsClassifier,17,16,18,16,17,18,18
1NeighborsClassifier,19,18,19,18,20,18,19
SGDClassifier,22,18,24,21,22,23,15
AdaBoost,13,13,12,13,14,13,9.3
SVC-rbf,20,9.4,19,25,19,21,14
GaussianNB,27,20,29,30,26,28,26
BernoulliNB,14,8.2,15,15,9.1,15,15
GradientBoost,13,10,12,15,15,11,13


In [44]:
df = pd.DataFrame(np.array(tutal),
                  columns=['avg','S I', 'M I', 'L I','S I', 'M F', 'L F'],index=clfNames)
df.round
df.astype(object)
df.to_csv('DDuplicate.csv')

,avg,S I,M I,L I,S I,M F,L F
RandomForestClassifier,22(+-15),15(+-12),22(+-14),33(+-16),21(+-16),22(+-14),26(+-16)
KNeighborsClassifier,19(+-17),15(+-16),17(+-17),27(+-16),17(+-16),21(+-17),16(+-18)
1NeighborsClassifier,21(+-19),15(+-17),19(+-18),32(+-17),20(+-20),23(+-18),18(+-19)
SGDClassifier,13(+-21),8(+-17),12(+-23),21(+-21),14(+-22),13(+-23),11(+-15)
AdaBoost,13(+-12),14(+-13),11(+-11),12(+-13),13(+-13),13(+-12),10(+-9)
SVC-rbf,13(+-19),4(+-9),12(+-19),25(+-24),12(+-19),15(+-21),7(+-13)
GaussianNB,32(+-27),31(+-20),28(+-28),39(+-30),32(+-26),31(+-28),33(+-25)
BernoulliNB,10(+-13),5(+-8),11(+-15),13(+-14),6(+-9),12(+-15),10(+-14)
GradientBoost,15(+-13),9(+-10),15(+-12),22(+-14),17(+-15),14(+-11),12(+-12)


In [2]:
#Three types of feature addition
func = 'cvScoreFeatures6Laptop'
func2 = 'cvScoreFeatures5Duration'
func3 = 'cvScoreFeatures4Duration'
clfName = clfNames[0]
amountList = [1.0]
funcMan = DoneFeatureMan(func,clfName,amountList)
func2Man = DoneFeatureMan(func2,clfName,amountList)
func3Man = DoneFeatureMan(func3,clfName,amountList)
intersect = []
for did in funcMan:
    if did in func2Man:
        if did in func3Man:
            intersect.append(did)
didList = []
for did in intersect:
    if did in NonCat:
        didList.append(did)
scores = []
scores2 = []
scores3 = []
amountList = [1]
for i,did in enumerate(didList):
    scores.append([])
    scores2.append([])
    scores3.append([])
    for clfName in clfNames:
        score1T = []
        score2T = []
        score3T = []
        for amount in amountList:
            if not checkForExist(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))) or not checkForExist(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))):
                print(func,clfName,amount,did)
            if did in Cat:
                score3T.append(read_features('cvScoreFeatures4Fixed',clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            else:
                score3T.append(read_features(func3,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score1T.append(read_features(func,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            score2T.append(read_features(func2,clfName,did,round(amount*(readDict(did)['NumberOfFeatures']-1))))
            
        scores[i].append(score1T)
        scores2[i].append(score2T)
        scores3[i].append(score3T)
scoreGD2 = []
scoreGID3 = []
scoreGR4 = []
ratio = []
for i in clfNames:
    scoreGD2.append([[] for i in range(len(amountList)+1)])
    scoreGID3.append([[] for i in range(len(amountList)+1)])
    scoreGR4.append([[] for i in range(len(amountList)+1)])
    ratio.append([])
MaxClass = []
for k,x in enumerate(scores):
    for j in range(0,len(x)):    
        for i in range(0,len(x[j])):
            scoreGD2[j][0].append(scores[k][j][i][0])
            scoreGD2[j][i+1].append(scores[k][j][i][1])
            scoreGID3[j][0].append(scores2[k][j][i][0])
            scoreGID3[j][i+1].append(scores2[k][j][i][1])
            scoreGR4[j][0].append(scores3[k][j][i][0])
            scoreGR4[j][i+1].append(scores3[k][j][i][1])

In [3]:
lossA = []
for k,temp in enumerate([scoreGR4]):
    lossA.append([])
    for i,item in enumerate(temp):
        lossA[k].append(100*(sum(temp[i][0])/len(temp[i][0])-sum(temp[i][1])/len(temp[i][1])))
argA = []
for i,clfName in enumerate(clfNames):
    argA.append([])
    for k, item in enumerate(lossA):
        argA[i].append(item[i])
stds = []
for i,item in enumerate(clfNames):
    stds.append([])
    for k in range(1):
        stds[i].append([])
for k,temp in enumerate([scoreGR4]):
    for i,item in enumerate(temp):#clf
        for j,itemz in enumerate(temp[i][0]):
            stds[i][k].append(temp[i][0][j]-temp[i][1][j])
diffA = []
for i,item in enumerate(stds):
    diffA.append([])
    for k,itemz in enumerate(stds[i]):
        diffA[i].append(100*np.array(stds[i][k]).std())

In [7]:
diffA

[[4.5183141183025866],
 [16.297624041821148],
 [17.478585627746476],
 [5.9830165750612236],
 [1.7571283314364312],
 [5.6911555288015894],
 [0.30176100443629439],
 [1.6396552481459488],
 [2.9344584322538441]]